In [1]:
!pip install dotenv --quiet

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env

embedding_model_name = "text-embedding-ada-002"
model_name = "gpt-4o-mini"


True

In [3]:
os.getenv("AZURE_OPENAI_ENDPOINT")

'https://user10-ai-service.openai.azure.com/'